In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import time


In [2]:
# Define the convolutional neural network
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [3]:
# Hyperparameters
num_classes = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.001


In [4]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Define model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes).to(device)
print(next(model.parameters()).device)

cuda:0


In [6]:
# Initialize the model, loss function, and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train():
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)  # Move to GPU
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


if __name__ == "__main__":
    import time

    # Measure training time
    start_time = time.time()
    train()
    end_time = time.time()

    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    # Measure inference time
    start_time = time.time()
    test()
    end_time = time.time()

    inference_time = end_time - start_time
    print(f"Inference time: {inference_time:.2f} seconds for the entire test set")

    per_sample_inference_time = inference_time / len(test_dataset)
    print(f"Inference time per sample: {per_sample_inference_time:.6f} seconds")

Epoch [1/10], Step [100/938], Loss: 0.2124
Epoch [1/10], Step [200/938], Loss: 0.2863
Epoch [1/10], Step [300/938], Loss: 0.0912
Epoch [1/10], Step [400/938], Loss: 0.0690
Epoch [1/10], Step [500/938], Loss: 0.1544
Epoch [1/10], Step [600/938], Loss: 0.0965
Epoch [1/10], Step [700/938], Loss: 0.0846
Epoch [1/10], Step [800/938], Loss: 0.1196
Epoch [1/10], Step [900/938], Loss: 0.0499
Epoch [2/10], Step [100/938], Loss: 0.0198
Epoch [2/10], Step [200/938], Loss: 0.0505
Epoch [2/10], Step [300/938], Loss: 0.0458
Epoch [2/10], Step [400/938], Loss: 0.0541
Epoch [2/10], Step [500/938], Loss: 0.0420
Epoch [2/10], Step [600/938], Loss: 0.1412
Epoch [2/10], Step [700/938], Loss: 0.0294
Epoch [2/10], Step [800/938], Loss: 0.0692
Epoch [2/10], Step [900/938], Loss: 0.0014
Epoch [3/10], Step [100/938], Loss: 0.0246
Epoch [3/10], Step [200/938], Loss: 0.0499
Epoch [3/10], Step [300/938], Loss: 0.0016
Epoch [3/10], Step [400/938], Loss: 0.0455
Epoch [3/10], Step [500/938], Loss: 0.0134
Epoch [3/10

In [7]:
# Save the model to a file
model_path = "simple_cnn.pth"
torch.save(model.state_dict(), model_path)

# Measure the size of the model file
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Convert bytes to MB
print(f"Model size: {model_size:.2f} MB")


Model size: 1.61 MB
